<div style="text-align: center; line-height: 0.2;">
    <h1><b>ADS 502: Introduction to Data Mining</b></h1>
    <h1><b>Final Project, Spring 2025</b></h1>
    <h2><b>Team One</b></h2>
</div>

### Team One is
    * Mark Villanueva
    * Paul Matta
    * Katie Kimberling

## Data and Dataset Overview

### Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from IPython.display import Image 
import plotly.graph_objects as go
from pandas.api.types import is_numeric_dtype
import warnings
warnings.filterwarnings('ignore')

### Import the training Tanzania water csv files and obtain overview of all of the attributes and datatypes

In [ ]:
pump_train_labels = pd.read_csv("https://raw.githubusercontent.com/KatieKimberling/ads-502-team-project/refs/heads/main/Pump_Train_Labels.csv")
print("The pump train LABELS dataset contains:") 
print(pump_train_labels.describe())
pump_train_labels.info()

pump_train_values = pd.read_csv("https://raw.githubusercontent.com/KatieKimberling/ads-502-team-project/refs/heads/main/Pump_Train_Values.csv")
print("The pump train VALUES dataset contains:") 
print(pump_train_values.describe())
pump_train_values.info()

### Merge the pump_train_values dataframe with the pump_train_labels dataframe to make one supervised training dataframe

In [ ]:
ptrain = pd.merge(pump_train_values, pump_train_labels, on='id', how='inner')
ptrain.head()

## Adjust date_recorded column to separate year from month from date

In [ ]:
ptrain['date_recorded'] = pd.to_datetime(ptrain['date_recorded'])
ptrain['year_recorded'] = ptrain['date_recorded'].dt.year
ptrain['month_recorded'] = ptrain['date_recorded'].dt.month

ptrain = ptrain.drop(['date_recorded'], axis=1)
ptrain.head()

In [ ]:
ptrain['status_group'].value_counts()

## Obtain, explore and handle missing values.

#### When prompted with "I want to show a list or grid or visual of how many missing values there are by variable" the ChatGPT-generated text indicated some of the following code to help me write a function to do so (OpenAI, 2025).

In [ ]:
# Create function to identify missing values in all variables in dataframe

def missing_value_summary(ptrain):
        
    # Calculate missing values and percentages
    
    missing_values = ptrain.isnull().sum()
    missing_percent = (missing_values / len(ptrain)) * 100
    
    # Create a summary DataFrame
    
    missing_summary = pd.DataFrame({
        'Missing Values': missing_values,
        'Percentage (%)': missing_percent
    }).sort_values(by='Missing Values', ascending=False)
    
    # Filter out variables without any missing values (i.e. only keep in those variables WITH missing values)
    
    missing_summary = missing_summary[missing_summary['Missing Values'] > 0]
    
    # Display the summary
    
    print(missing_summary)

    # Plot missing values as a bar chart
    
    if not missing_summary.empty:
        plt.figure(figsize=(10, 6))
        missing_summary['Missing Values'].plot(kind='barh', color='green', edgecolor='black')
        plt.xlabel("Count of Missing Values")
        plt.ylabel("Variables")
        plt.title("Missing Values by Variable")
        plt.gca().invert_yaxis()  # Invert y-axis for better readability
        plt.show()
    else:
        print("No missing values in the dataset!")

    return missing_summary  # Return the summary DataFrame

missing_summary = missing_value_summary(ptrain) 

## Replace missing values for permit and public_meeting with "unknown"

In [ ]:
ptrain['permit'] = ptrain['permit'].fillna('unknown')
ptrain['public_meeting'] = ptrain['public_meeting'].fillna('unknown')

ptrain['permit'].value_counts()

## Obtain discrete different levels of nominal variables with their counts

In [ ]:
def categorical_unique_counts(ptrain, categorical_vars):
     
    unique_counts = {col: ptrain[col].nunique() for col in categorical_vars}
    unique_counts_ptrain = pd.DataFrame.from_dict(unique_counts, orient='index', columns=['Unique Count'])
    
    return unique_counts_ptrain

categorical_vars = ["scheme_name", "scheme_management", "installer", "funder", "public_meeting", "permit", "subvillage", "wpt_name", "status_group"]
unique_counts_summary = categorical_unique_counts(ptrain, categorical_vars)

# Display the summary
print(unique_counts_summary)

In [ ]:
# Get the top 10 funders
top_funders = ptrain['funder'].value_counts().head(10)
print("Top 10 Funders:\n", top_funders)

# Get the top 10 installers
top_installers = ptrain['installer'].value_counts().head(10)
print("\nTop 10 Installers:\n", top_installers)

# Get the top 10 subvillages
top_subvillage = ptrain['subvillage'].value_counts().head(10)
print("\nTop 10 Subvillages:\n", top_subvillage)

# Get the top 10 scheme_names
top_scheme_names = ptrain['scheme_name'].value_counts().head(10)
print("\nTop 10 Scheme Names:\n", top_scheme_names)

## Replace missing values for funder and installer with "other"

In [ ]:
ptrain['funder'] = ptrain['funder'].fillna('other')
ptrain['installer'] = ptrain['installer'].fillna('other')

In [ ]:
# Count the occurrences of each category in the scheme_management  column
scheme_mgt_counts = ptrain['scheme_management'].value_counts()

# Create a horizontal bar chart of scheme_management with the bars colored red
plt.barh(scheme_mgt_counts.index, scheme_mgt_counts.values, color=['green'])

# Add labels and title
plt.ylabel('Scheme Management')
plt.xlabel('Counts')
plt.title('Bar Chart of Scheme Management')

# Show the plot
plt.show()

In [ ]:
# See if scheme management has different status_group counts
crosstab_01=pd.crosstab(ptrain['scheme_management'], ptrain['status_group'])
crosstab_01.plot(kind='barh', stacked = True, color=['green', 'yellow', 'red'])
plt.title('Bar Chart of Scheme Management with Status Group Overlay')
plt.show()

In [ ]:
# Normalized bar graph of "scheme_management" with "status_group" overlay - best practices is to always show the non-normalized bar plot with the normalized one, and the former is shown above.
crosstab_norm = crosstab_01.div(crosstab_01.sum(1), axis=0)
crosstab_norm.plot(kind='barh', stacked = True, color=['green', 'yellow', 'red'])
plt.title('Normalized Bar Chart of Scheme Management with Status Group Overlay')
plt.legend(title="Status Group", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# Deleting records with missing values for scheme_management
ptrain.dropna(subset=["scheme_management"], inplace=True)
ptrain.info()

## Examine categorical variables without missing values

In [ ]:
def cat_unique_counts(ptrain, cat_vars):
     
    unique_counts = {col: ptrain[col].nunique() for col in cat_vars}
    unique_counts_ptrain = pd.DataFrame.from_dict(unique_counts, orient='index', columns=['Unique Count'])
    
    return unique_counts_ptrain

cat_vars = ["basin", "region", "lga", "ward", "extraction_type", "extraction_type_group", "extraction_type_class", "management", "management_group", "payment", "payment_type", "water_quality", "quality_group", "quantity", "quantity_group", "source", "source_type", "source_class", "waterpoint_type", "waterpoint_type_group"]
unique_counts_summary = cat_unique_counts(ptrain, cat_vars)

# Display the summary
print(unique_counts_summary)

In [ ]:
def value_counts_cat_col(ptrain):

    for col in ptrain.select_dtypes(include='object').columns:
        print(f"Value counts for column '{col}':")
        print(ptrain[col].value_counts())
        print("-" * 20) # Separator for better readability

value_counts_cat_col(ptrain)

In [ ]:
# See if public_meeting has different status_group counts
crosstab_02=pd.crosstab(ptrain['public_meeting'], ptrain['status_group'])
crosstab_02.plot(kind='barh', stacked = True, color=['green', 'yellow', 'red'])
plt.title('Bar Chart of Public Meeting with Status Group Overlay')
plt.show()

In [ ]:
# Normalized bar graph of "public_meeting" with "status_group" overlay
crosstab_norm = crosstab_02.div(crosstab_02.sum(1), axis=0)
crosstab_norm.plot(kind='barh', stacked = True, color=['green', 'yellow', 'red'])
plt.title('Normalized Bar Chart of Public Meeting with Status Group Overlay')
plt.legend(title="Status Group", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# See if permit has different status_group counts
crosstab_03=pd.crosstab(ptrain['permit'], ptrain['status_group'])
crosstab_03.plot(kind='barh', stacked = True, color=['green', 'yellow', 'red'])
plt.title('Bar Chart of Permit with Status Group Overlay')
plt.show()

In [ ]:
# Normalized bar graph of "permit" with "status_group" overlay
crosstab_norm = crosstab_03.div(crosstab_03.sum(1), axis=0)
crosstab_norm.plot(kind='barh', stacked = True, color=['green', 'yellow', 'red'])
plt.title('Normalized Bar Chart of Permit with Status Group Overlay')
plt.legend(title="Status Group", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# See if quantity has different status_group counts
crosstab_04=pd.crosstab(ptrain['quantity'], ptrain['status_group'])
crosstab_04.plot(kind='barh', stacked = True, color=['green', 'yellow', 'red'])
plt.title('Bar Chart of Quantity with Status Group Overlay')
plt.show()

In [ ]:
# Normalized bar graph of "quantity" with "status_group" overlay
crosstab_norm = crosstab_04.div(crosstab_04.sum(1), axis=0)
crosstab_norm.plot(kind='barh', stacked = True, color=['green', 'yellow', 'red'])
plt.title('Normalized Bar Chart of Quantity with Status Group Overlay')
plt.legend(title="Status Group", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# See if water quality has different status_group counts
crosstab_05=pd.crosstab(ptrain['quality_group'], ptrain['status_group'])
crosstab_05.plot(kind='barh', stacked = True, color=['green', 'yellow', 'red'])
plt.title('Bar Chart of Water Quality with Status Group Overlay')
plt.show()

In [ ]:
# Normalized bar graph of "quality_group" with "status_group" overlay
crosstab_norm = crosstab_05.div(crosstab_05.sum(1), axis=0)
crosstab_norm.plot(kind='barh', stacked = True, color=['green', 'yellow', 'red'])
plt.title('Normalized Bar Chart of Water Quality Group with Status Group Overlay')
plt.legend(title="Status Group", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# View basin
basin = ptrain['basin'].value_counts().head(9)
print("Basin:\n", basin)

# View regions
region = ptrain['region'].value_counts().head(21)
print("\nRegions:\n", region)

# Get the top 10 lga
top_lga = ptrain['lga'].value_counts().head(10)
print("\nTop 10 lga:\n", top_lga)

# Get the top 10 wards
top_ward = ptrain['ward'].value_counts().head(10)
print("\nTop 10 Wards:\n", top_ward)

In [ ]:
# See if basin has different status_group counts
crosstab_06=pd.crosstab(ptrain['basin'], ptrain['status_group'])
crosstab_06.plot(kind='barh', stacked = True, color=['green', 'yellow', 'red'])
plt.title('Bar Chart of Basin with Status Group Overlay')
plt.show()

In [ ]:
# Normalized bar graph of "basin" with "status_group" overlay
crosstab_norm = crosstab_06.div(crosstab_06.sum(1), axis=0)
crosstab_norm.plot(kind='barh', stacked = True, color=['green', 'yellow', 'red'])
plt.title('Normalized Bar Chart of Basin with Status Group Overlay')
plt.legend(title="Status Group", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

### Obtain summary statistics of all numeric variables in ptrain dataframe and plot their histograms

In [ ]:
for col in ptrain.columns:
    if is_numeric_dtype(ptrain[col]):
        print('%s:' % (col))
        print('\t Mean = %.2f' % ptrain[col].mean())
        print('\t Median = %.2f' % ptrain[col].median())
        print('\t Standard deviation = %.2f' % ptrain[col].std())
        print('\t Minimum = %.2f' % ptrain[col].min())
        print('\t Maximum = %.2f' % ptrain[col].max())
        print('\t Missing value count = %.2f' % ptrain[col].isnull().sum())

def plot_numerical_histograms(ptrain):
    numerical_cols = ptrain.select_dtypes(include=['number']).columns
    ptrain[numerical_cols].hist(figsize = (10,8), color="red")
    plt.tight_layout()
    plt.show()
plot_numerical_histograms(ptrain)

## Calculate the correlation matrix

In [ ]:
# Select only numeric columns
numeric_df = ptrain.select_dtypes(include=["int64", "float64"])

# Calculate the correlation matrix
corr_matrix = numeric_df.corr(method='pearson')

# Visualize the correlation matrix using a heatmap
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

### Raw Box and Whisker Plots of Population by Region (Waskom, 2024)

In [ ]:
g = sns.catplot(data=ptrain, x="population", y="region", palette="husl", kind="boxen")
g.fig.suptitle("Population by Region (Raw Data)")
plt.tight_layout()

#### There are significant outliers on the upper end in each region, which is skewing the data. Outliers were identified as being beyond 1.5*IQR and removed from the variable population by region as shown below.

### Detect and recode outliers using IQR method (Geeks for Geeks, 2025)

In [ ]:
# Function to identify and remove outliers using IQR

def remove_outliers_iqr(ptrain, columns):
    for col in columns:
        Q1 = ptrain[col].quantile(0.25)
        Q3 = ptrain[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Keep only values within the bounds
        ptrain = ptrain[(ptrain[col] >= lower_bound) & (ptrain[col] <= upper_bound)]
    
    return ptrain
    
# List of numerical variables to check for outliers: population, amount_tsh and gps_height
clean_columns = ["population", "amount_tsh", "gps_height"]
ptrain_cleaned = remove_outliers_iqr(ptrain, clean_columns)

# Apply outlier removal to each region
print(ptrain_cleaned.describe())

In [ ]:
g = sns.catplot(data=ptrain_cleaned, x="population", y="basin", palette="husl", kind="boxen")
g.fig.suptitle("Population by Basin After Outliers Removed")
plt.tight_layout()

In [ ]:
sns.scatterplot(data=ptrain_cleaned, x="longitude", y="latitude")
plt.title("Latitudes and Longitudes Raw Data")
plt.tight_layout()

## Adjust latitudes and longitudes to fit Tanzania

In [ ]:
# Define valid ranges
valid_lat_range = (-15, 5)
valid_lon_range = (25, 42)

# Filter outliers
ptrain_cleaned = ptrain_cleaned[ptrain_cleaned['latitude'].between(*valid_lat_range) & ptrain_cleaned['longitude'].between(*valid_lon_range)]
sns.scatterplot(data=ptrain_cleaned, x="longitude", y="latitude")
plt.title("Adjusted Latitudes and Longitudes")

## Create map of water pumps in Tanzania by functional status

#### In order to create a map of the water pumps in Tanzania by function status, OpenAI's ChatGPT was queried to complete this task on March 28, 2025. The code below was generated entirely by ChatGPT and simply modified to fit this project's dataframe. (OpenAI, 2025)

In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Load and filter data (ensure only Tanzania data)
tanzania_bounds = {
    "lat_min": -11.7, "lat_max": -0.9,
    "lon_min": 29.2, "lon_max": 40.4
}

# Assuming 'ptrain_cleaned' is your DataFrame
filtered_df = ptrain_cleaned[
    (ptrain_cleaned["latitude"].between(tanzania_bounds["lat_min"], tanzania_bounds["lat_max"])) &
    (ptrain_cleaned["longitude"].between(tanzania_bounds["lon_min"], tanzania_bounds["lon_max"]))
]

# Create a map centered on Tanzania with a smaller file size
tanzania_map = folium.Map(
    location=[-6.369028, 34.888822],
    zoom_start=7,
    tiles="CartoDB positron"  # Lighter map to reduce file size
)

# Define colors for pump status
status_colors = {
    "functional": "green",
    "functional needs repair": "yellow",
    "non functional": "red"
}

# Add pump markers (without clustering for smaller size)
for _, row in filtered_df.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=3,  # Smaller marker to save space
        color=status_colors.get(row["status_group"], "blue"),
        fill=True,
        fill_color=status_colors.get(row["status_group"], "blue"),
        fill_opacity=0.5,  # Lower opacity for better visibility
        popup=row["status_group"]  # Smaller popup
    ).add_to(tanzania_map)

# Add a legend (this is a custom HTML/CSS legend)
legend_html = """
<div style="position: fixed; bottom: 50px; left: 50px; width: 200px; height: 100px;
background-color: white; z-index:9999; font-size:14px; padding: 10px; border-radius:5px;">
<b>Pump Status</b><br>
<i class="fa fa-circle" style="color:green"></i> Functional<br>
<i class="fa fa-circle" style="color:orange"></i> Functional Needs Repair<br>
<i class="fa fa-circle" style="color:red"></i> Non Functional
</div>
"""
tanzania_map.get_root().html.add_child(folium.Element(legend_html))
tanzania_map

## Drop all unnecessary variables/features/attributes/columns

In [ ]:
ptrain_cleaned = ptrain_cleaned.drop(['scheme_name', 'month_recorded', 'district_code', 'region_code', 'public_meeting', 'permit', 'subvillage', 'wpt_name', 'num_private', 'region', 'lga', 'ward', 'recorded_by', 'extraction_type', 'extraction_type_class', 'management', 'management_group', 'payment_type', 'water_quality', 'quantity_group', 'source', 'source_class', 'waterpoint_type_group', 'id'], axis=1)
ptrain_cleaned.info()

### References

Geeks for Geeks. (2025, Jan 2). *Detect and remove the outliers using Python.* Geeks for Geeks. [GeeksforGeeks](https://www.geeksforgeeks.org/detect-and-remove-the-outliers-using-python/)
 
Kahn, R. (2020). Pump it up: Data mining the water table [Map]. GitHub. [Taarifa Pump Github](https://github.com/rkhan15/taarifa-pump)

OpenAI. (2025). ChatGPT (Mar 26 version) [Large language model]. https://chat.openai.com/chat

OpenAI. (2025). ChatGPT (Mar 28 version) [Large language model]. https://chat.openai.com/chat

Waskom, M. (2024). *Visualizing categorical data.* Seaborn. [Seaborn visualizations](https://seaborn.pydata.org/tutorial/categorical.html)